<a href="https://colab.research.google.com/github/ok2000/gvr-unity-sdk/blob/master/%E5%9C%96%E7%89%87%E5%88%86%E9%A1%9E%E8%88%87%E5%9C%96%E7%89%87%E6%8A%93%E5%8F%96%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##圖片分類篩選與圖片爬蟲抓取程式

prompt：

創建一個Python程式，專門在Google Colab環境中執行，功能用於圖片處理和下載。程式具有以下特點和功能：

1.設計為在Google Colab notebook中執行，充分利用Colab的特性。

2.自動安裝和導入所有必要的套件庫，如果需要特殊版本的套件庫，請說明安裝命令。

3.使用Google Colab的文件系統接口來操作Google Drive，包括掛載Drive和訪問/創建文件夾。

4.提供兩個主要功能：
a) 圖片分類：根據圖片像素大小將Google Drive中的圖片分類到不同資料夾。

b) 圖片抓取：從DuckDuckGo搜索並下載指定數量的圖片到Google Drive。

5.使用Selenium WebDriver或其他相容於Google Colab的套件進行網頁資料爬蟲，包含Colab環境中設置Chrome WebDriver的代碼。

6.需要架構一個主選單循環，允許用戶多次選擇執行不同功能或退出程式。

7.圖片分類功能應接受Google Drive中的輸入和輸出資料夾路徑。

8.圖片抓取功能應接受搜索關鍵字和下載數量，並將圖片保存到指定的Google Drive資料夾。

9.包含錯誤處理和用戶友好的提示訊息。

10.每行代碼都應有繁體中文註解，並且解釋其功能。

11.考慮到Colab的執行環境可能會重置，請確保代碼可以多次執行而不會出錯。

12.如果有任何需要用戶手動設置的步驟（例如允許訪問Google Drive），請在代碼中提供清晰的指示。

請提供完整的、可以直接在Google Colab環境中運行的Python程式，包括所有必要的設置步驟和函數定義。確保程式遵循Python的最佳實踐效能，並針對Colab環境進行優化。

In [ ]:
import os  # 引入操作系統模組，用於檔案和目錄操作
from PIL import Image  # 引入PIL模組，用於處理圖片
import shutil  # 引入shutil模組，用於檔案移動
from selenium import webdriver  # 引入Selenium模組，用於自動化瀏覽器操作
from selenium.webdriver.common.by import By  # 引入By模組，用於定位網頁元素
from selenium.webdriver.chrome.options import Options  # 引入Chrome選項模組，用於設定無頭模式
import time  # 引入時間模組，用於控制延遲
import requests  # 引入requests模組，用於發送HTTP請求
import sys  # 引入系統模組，用於處理系統相關操作
from google.colab import drive  # 引入Google雲端硬碟模組，用於掛載Google Drive

# 設定ChromeDriver路徑
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')  # 將chromedriver路徑加入系統環境變數

# 創建資料夾如果不存在
def create_folder_if_not_exists(folder):
    if not os.path.exists(folder):  # 如果資料夾不存在
        os.makedirs(folder)  # 則創建該資料夾

# 圖片分類函數，根據像素大小分類
def classify_images(input_folder, output_folder_A, output_folder_B, max_pixels=1024*1024):
    create_folder_if_not_exists(output_folder_A)  # 確保低像素分類資料夾存在
    create_folder_if_not_exists(output_folder_B)  # 確保高像素分類資料夾存在
    for filename in os.listdir(input_folder):  # 遍歷輸入資料夾中的每個檔案
        filepath = os.path.join(input_folder, filename)  # 獲取完整檔案路徑
        if os.path.isfile(filepath):  # 如果是檔案
            try:
                with Image.open(filepath) as img:  # 使用PIL打開圖片
                    width, height = img.size  # 獲取圖片寬度和高度
                    if width * height < max_pixels:  # 檢查圖片像素是否小於閾值
                        shutil.copy(filepath, os.path.join(output_folder_A, filename))  # 小於閾值則移動到低像素資料夾
                    else:
                        shutil.copy(filepath, os.path.join(output_folder_B, filename))  # 大於或等於閾值則移動到高像素資料夾
            except Exception as e:  # 如果出現錯誤
                print(f"處理 {filename} 時發生錯誤: {e}")  # 輸出錯誤訊息
    print("圖片分類完成。")  # 輸出完成訊息

# 初始化WebDriver
def setup_driver():
    options = Options()  # 建立Chrome選項物件
    options.add_argument('--headless')  # 無頭模式，不顯示瀏覽器
    options.add_argument('--no-sandbox')  # 禁用沙盒模式
    options.add_argument('--disable-dev-shm-usage')  # 禁用共享記憶體
    driver = webdriver.Chrome(options=options)  # 初始化ChromeDriver
    return driver  # 返回Driver物件

# 下載圖片函數，將圖片保存到指定的資料夾中
processed_urls = set()  # 使用集合儲存已處理過的圖片URL，避免重複下載

def download_image(image_url, folder_path):
    if image_url in processed_urls:  # 如果URL已經處理過
        print("URL已處理過，跳過。")  # 輸出訊息並跳過
        return  # 不再處理該URL

    response = requests.get(image_url)  # 發送HTTP請求下載圖片
    if response.status_code == 200:  # 如果響應成功
        image_content = response.content  # 獲取圖片內容

        # 生成唯一的檔名
        counter = 1  # 初始化計數器
        while True:  # 開始循環
            image_name = f"image_{counter}.jpg"  # 生成檔名
            image_path = os.path.join(folder_path, image_name)  # 組合完整的圖片儲存路徑
            if not os.path.exists(image_path):  # 如果檔案不存在
                break  # 跳出循環
            counter += 1  # 增加計數器

        with open(image_path, 'wb') as f:  # 以二進制模式打開檔案，準備寫入圖片
            f.write(image_content)  # 寫入圖片內容到檔案
        processed_urls.add(image_url)  # 將URL加入已處理集合
        print(f"圖片已下載並保存到 {image_path}")  # 輸出成功訊息
        return True  # 返回成功標誌
    else:
        print(f"下載圖片失敗，狀態碼：{response.status_code}")  # 如果下載失敗，輸出錯誤狀態碼
        return False  # 返回失敗標誌

# 抓取圖片函數，根據使用者指定的數量抓取圖片
def fetch_images(query, folder_path, max_images):
    driver = setup_driver()  # 初始化ChromeDriver
    driver.get(f'https://duckduckgo.com/?q={query}&iax=images&ia=images')  # 打開DuckDuckGo圖片搜索頁面
    time.sleep(5)  # 等待頁面加載

    create_folder_if_not_exists(folder_path)  # 確保圖片存儲資料夾存在

    num_images_fetched = 0  # 計算抓取的圖片數量
    last_height = driver.execute_script("return document.body.scrollHeight")  # 獲取初始頁面高度

    while num_images_fetched < max_images:  # 當抓取的圖片數量小於目標時
        # 滾動到頁面底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 等待新內容加載

        # 獲取新的頁面高度並與上一次的進行比較
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 如果頁面高度沒有變化，表示已經到達底部
            break  # 跳出循環
        last_height = new_height  # 更新頁面高度

        # 找到所有圖片元素
        image_elements = driver.find_elements(By.CSS_SELECTOR, "img.tile--img__img")

        for image in image_elements:  # 遍歷找到的圖片元素
            if num_images_fetched >= max_images:  # 如果已經達到要抓取的圖片數量
                break  # 停止抓取

            try:
                image_src = image.get_attribute('src')  # 獲取圖片URL
                if image_src and image_src not in processed_urls:  # 如果URL存在且未處理過
                    if download_image(image_src, folder_path):  # 下載圖片
                        num_images_fetched += 1  # 增加已抓取圖片數量
                        print(f"已下載 {num_images_fetched} 張圖片")  # 輸出進度
            except Exception as e:  # 如果出現錯誤
                print(f"處理圖片時發生錯誤: {e}")  # 輸出錯誤訊息
                continue  # 跳過該圖片，繼續處理下一張圖片

    driver.quit()  # 抓取完成後關閉瀏覽器
    print(f"圖片抓取完成，共抓取了 {num_images_fetched} 張圖片。")  # 輸出完成訊息

# 主程式，用於執行圖片分類和抓取功能
def main():
    drive.mount('/content/drive')  # 掛載Google雲端硬碟

    while True:  # 無限循環，直到使用者選擇退出
        print("\n請選擇要執行的功能：")  # 提示使用者選擇功能
        print("1. 圖片分類")  # 選項1：分類圖片
        print("2. 圖片抓取")  # 選項2：抓取圖片
        print("3. 結束程式")  # 選項3：結束程式
        choice = input("輸入選項 (1, 2 或 3): ")  # 接收使用者選擇

        if choice == "1":  # 如果選擇1
            input_folder = input("請輸入圖片來源資料夾路徑 (在Google Drive中): ")  # 提示輸入來源資料夾
            output_folder_A = input("請輸入像素不足圖片存放資料夾路徑 (在Google Drive中): ")  # 輸入低像素圖片資料夾
            output_folder_B = input("請輸入像素足夠圖片存放資料夾路徑 (在Google Drive中): ")  # 輸入高像素圖片資料夾
            classify_images(input_folder, output_folder_A, output_folder_B)  # 執行圖片分類
        elif choice == "2":  # 如果選擇2
            query = input("請輸入要抓取圖片的關鍵字: ")  # 提示輸入圖片抓取的關鍵字
            folder_path = '/content/drive/My Drive/抓取的圖片'  # 設定雲端硬碟的存儲路徑
            try:
                max_images = int(input("請輸入要抓取的圖片數量: "))  # 提示輸入要抓取的圖片數量
            except ValueError:
                print("請輸入有效的數字，請重新選擇。")  # 如果輸入無效數字，提示重新選擇
                continue  # 繼續下一次循環

            fetch_images(query, folder_path, max_images)  # 執行圖片抓取
        elif choice == "3":  # 如果選擇3
            print("程式結束，謝謝使用。")  # 輸出結束訊息
            break  # 跳出循環，結束程式
        else:
            print("無效的選項，請重新選擇。")  # 如果輸入無效，提示重新選擇

# 呼叫主程式
if __name__ == "__main__":
    main()  # 執行主程式

ModuleNotFoundError: No module named 'selenium'